In [1]:
import smbus2
import time
from IPython.display import clear_output
import datetime

In [2]:
addr_sht = 0x44
addr_switch = 0x70
addr_sdcard = 0x71
addr_charger = 0x6a
addr_gauge = 0x55
addr_rtc = 0x51
addr_eeprom = 0x50
addr_eepromsn = 0x58
addr_altimet = 0x77

addresses = {
    'sht': addr_sht, 
    'switch': addr_switch, 
    'sdcard': addr_sdcard,
    'charger': addr_charger,
    'gauge': addr_gauge,
    'rtc': addr_rtc,
    'eeprom': addr_eeprom,
    'eepromsn': addr_eepromsn, 
    'altimet': addr_altimet}

In [3]:
def read_2(c, addr, reg):
    d = c.read_i2c_block_data(addr, reg, 2)
    #print([bin(x) for x in d])

    d = d[0]| d[1] << 8
    #print(bin(d))
    return d

def crop(number, start=0, end=16):
    # Vytvoření masky s jedničkami na pozicích od start do end
    mask = ((1 << (end - start + 1)) - 1) << start
    
    # Použití masky pro vybrání příslušných bitů
    result = (number & mask) >> start
    
    return result

def comp(data, length=16):

    if data & (1 << (length-1)):
        data = data ^ ((1 << length) - 1)
        data = -(data + 1)

    return data



In [4]:
try:
    c.close()
except Exception as e:
    print("na")

na


In [5]:
def con():
    return smbus2.SMBus(8)

c = con()

# I2C switch, switch internal I2C bus into USB (FT260) IO

In [6]:
c=con()

In [7]:
c.write_byte_data(addr_switch, 0x01, 0x03) # nastavit I2C switch do USB

In [8]:
hex(c.read_byte(addr_switch, 0x01))

'0xb'

In [9]:
import sys
def scan_i2c_bus(_bus):
  devices = []
  sys.stdout.write ('   ')
  for _address in range(16):
    sys.stdout.write (' %2x' % _address) # Print header
  
  for _address in range(128):
    if not _address % 16:
      sys.stdout.write ('\n%02x:' % _address) # Print address
    if 2 < _address < 120 : # Skip reserved addresses
      try:
        _bus.read_byte(_address)
        sys.stdout.write (' %02x' % _address) # Device address
        devices.append(_address)
      except:
        sys.stdout.write (' --') # No device detected
    else:
      sys.stdout.write ('   ') # Reserved

  sys.stdout.write ('\n')
  return devices

devices = scan_i2c_bus(c)
devices = set(devices)

print("\n\n")
for an in addresses:
  print(f"(0x{addresses[an]:02x}) - {an}: {addresses[an] in devices}" )
  #print(an, addressesav, av in devices)

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- --
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
40: -- -- -- -- 44 -- -- -- -- -- -- -- -- -- -- --
50: 50 51 -- -- -- 55 -- -- 58 -- -- -- -- -- -- --
60: -- -- -- -- -- -- -- -- -- -- 6a -- -- -- -- --
70: 70 71 -- -- -- -- -- --                        



(0x44) - sht: True
(0x70) - switch: True
(0x71) - sdcard: True
(0x6a) - charger: True
(0x55) - gauge: True
(0x51) - rtc: True
(0x50) - eeprom: True
(0x58) - eepromsn: True
(0x77) - altimet: False


# Nabíječka 

In [10]:
# Vypnout bateriovy modul
if 0:
    c.write_byte_data(addr_charger, 0x18, 0x0a)

In [11]:
print(bin(c.read_byte_data(addr_charger, 0x16)))


charge_current_ma = 500

c.write_byte_data(addr_charger, 0x10, 0x1f << 3)

c.write_byte_data(addr_charger, 0x02, int(charge_current_ma/40)<<5)
c.write_byte_data(addr_charger, 0x14, 0b00100110)
c.write_byte_data(addr_charger, 0x15, 0b00011101)
c.write_byte_data(addr_charger, 0x16, 0b10100000)
c.write_byte_data(addr_charger, 0x17, 0b01010110)
c.write_byte_data(addr_charger, 0x18, 0b00000000)
c.write_byte_data(addr_charger, 0x19, 0b00000001)


# NTC
c.write_byte_data(addr_charger, 0x1a, 0b10111111)

# ADC
c.write_byte_data(addr_charger, 0x26, 0b10001100)

0b10100001


In [12]:

while True:
    # Zapnout interni ADC prevodnik
    #print(bin(c.read_byte_data(addr_charger, 0x26)))
    #c.write_byte_data(addr_charger, 0x26, 0b10001100)
    
    clear_output()
    print("IBUS ADC", comp(crop(read_2(c, addr_charger, 0x28), 1, 16), 15) * 2, " mA")
    #print(" ")

    print("IBAT ADC", comp(crop(read_2(c, addr_charger, 0x2A), 2, 16), 14) * 4, " mA")
    #print(" ")

    print("VBUS ADC", crop(read_2(c, addr_charger, 0x2C), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VPMID ADC", crop(read_2(c, addr_charger, 0x2E), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VBAT ADC", crop(read_2(c, addr_charger, 0x30), 1, 13) * 1.99, " mV")
    #print(" ")

    print("VSYS ADC", crop(read_2(c, addr_charger, 0x32), 1, 13) * 1.99, " mV")
    #print(" ")

    #print("TS ADC", crop(read_2(c, addr_charger, 0x34), 0, 12) * 0.0961, " %")
    #print(" ")

    #print("TDIE ADC", crop(read_2(c, addr_charger, 0x36), 0, 12) * 0.5 - 40, " C")
    #print(" ")

    time.sleep(1)
    break


IBUS ADC 56  mA
IBAT ADC 0  mA
VBUS ADC 5037.93  mV
VPMID ADC 5037.93  mV
VBAT ADC 4177.01  mV
VSYS ADC 4200.89  mV


In [13]:
# Nastavit timery
c.write_byte_data(addr_charger, 0x15, 0b00001001)

In [14]:
d = c.read_byte_data(addr_charger, 0x1d)
print("Charger status 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1e)
print("Charger status 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1f)
print("Fault reg 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x20)
print("Charger flag 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x21)
print("Charger flag 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x22)
print("Fault flag 0: 0b{0:08b}".format(d))

Charger status 0: 0b00000000
Charger status 1: 0b00010100
Fault reg 0: 0b00000000
Charger flag 0: 0b00010001
Charger flag 1: 0b00001001
Fault flag 0: 0b00000001


In [15]:
voltage = c.read_i2c_block_data(0x55,0x08,2)
voltage = (voltage[1] << 8) | voltage[0]
print(f"Voltage {voltage} mV")

current = c.read_i2c_block_data(0x55,0x0a,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA (avg)")

current = c.read_i2c_block_data(0x55,0x10,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA")


d = c.read_word_data(0x55,0x04)
print(f"RemainingCapacity {d} mAh")


d = c.read_word_data(0x55,0x06)
print(f"Full Capacity {d} mAh")

d = c.read_word_data(0x55,0x0c)
print(f"Temp {d/10-273} C")


d = c.read_byte(0x55,0x02)
print(f"State: {d} %")


#d = c.read_ (0x55,0x04,2)
#d = (d[1] << 8) | d[0]
#print(f"CRemainingCapacity {d} mAh")

Voltage 98 mV
Current 0 mA (avg)
Current 0 mA
RemainingCapacity 0 mAh
Full Capacity 978 mAh
Temp 24.600000000000023 C
State: 22 %


In [16]:
import struct 
bus = c
address = 0x55

e=bus.read_i2c_block_data(address,0x0a,2)
(full_cap,)=struct.unpack('H', bytearray(e)[0:2])
print("Full Available Capacity is",full_cap, 'mA')

#Trying to write the nominal capacity
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)

bus.write_byte_data(address,0x00,0x13)
bus.write_byte_data(address,0x01,0x00)

#bus.write_byte_data(address,0x61,0x00)
#bus.write_byte_data(address,0x3e,0x52)
#bus.write_byte_data(address,0x3f,0x00)

bus.write_byte_data(address,0x4a,0x04) #writing new capacity
bus.write_byte_data(address,0x4b,0xb0) #writing new capacity

time.sleep(1)

#bus.write_byte_data(address,0x60,0x1f) #trying to write on BlockDataChecksum() 

time.sleep(1)

bus.write_byte_data(address,0x00,0x42)
bus.write_byte_data(address,0x01,0x00)

bus.write_byte_data(address,0x00,0x20)
bus.write_byte_data(address,0x01,0x00)

f=bus.read_i2c_block_data(address,0x3c,2) #address for design capacity
(des_cap,)=struct.unpack('H', bytearray(f)[0:2])
print(des_cap)


Full Available Capacity is 0 mA
1000


EEPROM + Serial number...

In [17]:
def read_serial_number(address):
    serial_number = []
    c.write_byte_data(address, 0x08, 0x00)
    for _ in range(16):
        serial_byte = c.read_byte(address)
        serial_number.append(serial_byte)
    
    result_number = 0
    for b in serial_number:
        result_number = (result_number << 8) | b

    return result_number

detector_serial_number = read_serial_number(addr_eepromsn)
print( hex(detector_serial_number) )


0x1290c00806a200925448a000a0000063


In [18]:

# Příkazy pro čtení teploty a vlhkosti
temperature_cmd = [0x24, 0x00]  # Příkaz pro čtení teploty v režimu High Precision
humidity_cmd = [0x24, 0x16]     # Příkaz pro čtení vlhkosti v režimu High Precision
serial_number_cmd = [0x37, 0x80]

# Funkce pro čtení dat ze senzoru
def sht_read_sensor_data(cmd):
    c.write_i2c_block_data(addr_sht, cmd[0], [cmd[1]])
    time.sleep(0.1)  # Počkejte na dokončení měření (0.1 sekundy)
    data = c.read_i2c_block_data(addr_sht, 0, 6)
    raw_temperature = (data[0] << 8) + data[1]
    raw_humidity = (data[3] << 8) + data[4]
    temperature = -45 + 175 * (raw_temperature / 65535.0)  # Výpočet teploty
    humidity = 100 * (raw_humidity / 65535.0)             # Výpočet vlhkosti
    return temperature, humidity


def sht_read_sn(cmd):
    c.write_i2c_block_data(addr_sht, cmd[0], [cmd[1]])
    data = c.read_i2c_block_data(addr_sht, 0, 6)
    print(data)
    serial_number = (data[0] << 24) | (data[1] << 16) | (data[3] << 8) | data[4]
    return serial_number

# Čtení teploty a vlhkosti ze senzoru
temperature, humidity = sht_read_sensor_data(temperature_cmd)
print("Teplota: {:.2f} °C".format(temperature))
print("Vlhkost: {:.2f} %".format(humidity))

sht_serial_number = sht_read_sn(serial_number_cmd)
print("Sériové číslo: 0x{:x}".format(sht_serial_number))


Teplota: 22.46 °C
Vlhkost: 46.03 %
[28, 164, 99, 49, 68, 62]
Sériové číslo: 0x1ca43144



# RTC

In [19]:
import datetime

In [20]:
def get_time():
    r01 = c.read_byte_data(addr_rtc, 0x01)
    r02 = c.read_byte_data(addr_rtc, 0x02)
    r03 = c.read_byte_data(addr_rtc, 0x03)
    r04 = c.read_byte_data(addr_rtc, 0x04)
    r06 = c.read_byte_data(addr_rtc, 0x06)
    r07 = c.read_byte_data(addr_rtc, 0x07)

    sec = ((r01 >> 4) & 0b111) * 10 + (r01 & 0b1111)
    minu= ((r02 >> 4) & 0b111) * 10 + (r02 & 0b1111)
    hour= ((r03 >> 4) & 0b11) * 10 + (r03 & 0b1111)
    day = ((r04 >> 4) & 0b11) * 10 + (r04 & 0b1111)
    mon = ((r06 >> 4) & 0b1) * 10 + (r06 & 0b1111)
    year= ((r07 >> 4) & 0b1111) * 10 + (r07 & 0b1111)+2000

    print(year, mon, day, hour, minu, sec)        
    return datetime.datetime(year, mon, day, hour, minu, sec)
    
def set_time(time):
    c.write_byte_data(addr_rtc, 0x01, (time.second % 10) | int(time.second/10)<<4 )
    c.write_byte_data(addr_rtc, 0x02, (time.minute % 10) | int(time.minute/10)<<4 )
    c.write_byte_data(addr_rtc, 0x03, (time.hour % 10) | int(time.hour/10)<<4 )
    c.write_byte_data(addr_rtc, 0x04, (time.day % 10) | int(time.day/10)<<4 )
    c.write_byte_data(addr_rtc, 0x06, (time.month % 10) | int(time.month/10)<<4 )
    c.write_byte_data(addr_rtc, 0x07, ((time.year-2000) % 10) | int((time.year-2000)/10)<<4 )

In [21]:
actual_time = datetime.datetime.utcnow()
detector_time = get_time()
print("Actual time", actual_time)
print("Detector time", detector_time)
print("Difference", detector_time-actual_time)

2000 1 1 0 46 29
Actual time 2023-12-11 21:22:36.709698
Detector time 2000-01-01 00:46:29
Difference -8746 days, 3:23:52.290302


In [22]:
actual_time = datetime.datetime.utcnow()
set_time(actual_time)

In [23]:
actual_time.second

38

In [24]:

time = actual_time

In [25]:
bin( (time.second % 10)<<4)

'0b10000000'

In [44]:
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
from datetime import datetime
from IPython.display import display, FileLink

# Funkce pro vytvoření a zobrazení PDF reportu s logem a názvem firmy
def create_and_display_pdf_report():
    # Název zařízení
    device_name = "BATDATUNIT01B"

    # Čas a datum vytvoření
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    # Název firmy
    company_name = "Universal Scientific Technologies s.r.o."  # Nahraďte názvem vaší firmy
    logo_path = "Logo_UST.png"

    # Vytvoření PDF obsahu
    filename = f"batdatunit01a_report_0x{detector_serial_number:0x}.pdf"
    pdf = canvas.Canvas(filename, pagesize=A4)
    pdf.setFont("Helvetica", 12)

    # Přidání loga a názvu firmy do hlavičky
    pdf.drawImage(logo_path, 80, 720, width=100, preserveAspectRatio=1)
    pdf.drawString(280, 765, company_name)
    pdf.drawString(280, 750, "www.ust.cz, support@ust.cz")

    pdf.drawString(80, 690, "Test report for " + device_name + f"    SN: 0x{detector_serial_number:0x}")
    pdf.drawString(80, 675, "Generated on " + current_time)
    
    pdf.drawString(80, 640, "I2C Devices:")
    y = 620


    #print(f"(0x{addresses[an]:02x}) - {an}: {addresses[an] in devices}" )
    for idev in addresses:
        addr = addresses[idev]
        pdf.drawString(100, y, f"{idev}")
        pdf.drawString(230, y, f"0x{addr:02x}")
        pdf.drawString(280, y, "Pass" if (addr in devices) else "Failed" )
        y -= 20

    pdf.drawString(80, 370, "Serial Numbers:")
    pdf.drawString(100, 350, f"Detector Serial Number:")
    pdf.drawString(280, 350, f"0x{detector_serial_number:0x}")
    pdf.drawString(100, 330, f"SHT3x Serial Number:")
    pdf.drawString(230, 330, f"0x{sht_serial_number:0x}")
    #pdf.drawString(70, 310,  "Current Temperature: " + current_temperature)

    # Uložení PDF
    pdf.save()

    # Zobrazit PDF v Jupyter Notebooku
    display(FileLink(filename))

# Volání funkce pro vytvoření a zobrazení PDF reportu
create_and_display_pdf_report()


/home/roman/repos/ust/AIRDOS04/sw/batdatunit01a_report_0x1290c00806a200925448a000a0000063.pdf